In [1]:
import pandas as pd
import multiprocessing as mp
import numpy as np

In [2]:
data = pd.read_csv('../matriksdata.csv')

In [3]:
# zaman sütunu datetime64 tipine dönüştürülüyor
data['date_time'] = pd.to_datetime(data.date_time)

In [25]:
# Aynı datetime'a sahip symbol'lerden timestamp_ns değeri yüksek olan seçiliyor.
data=data.groupby(['symbol','date_time']).apply(lambda x : last_time(x))
data.reset_index(drop=True,inplace = True)

CPU times: user 12 s, sys: 197 ms, total: 12.2 s
Wall time: 11.8 s


In [ ]:
# # Aynı datetime'a sahip symbol'lerin ortalaması alınıyor
# data = data.groupby(['symbol','date_time']).mean()
# data.reset_index(inplace=True)

In [ ]:
# mid_price hesaplanıyor
data['mid_price'] = (data['bid_price']+data['ask_price'])/2

In [ ]:
# symbol'ler sütunlara gelecek şekilde düzenleniyor
data = data.pivot(index='date_time',columns='symbol',values='mid_price')

In [ ]:
# Zaman seriler resample metodu kullanılarak, belirlenen koşula göre parçalara ayrılabilir.
# Burada 'd' parametresi kullanılarak veriler günlük olarak bölünüyor
# Diğer seçenekler için açıklama aşağıda
daily_data = data.resample('D')

In [ ]:
# # Veriler 5'er dakika olarak gösteriliyor.
# resample_data = data.resample('5Min',label='right',closed='right').last()

In [ ]:
# # NaN değerler bir önceki veri ile dolduruluyor
# resample_data = resample_data.fillna(method='ffill')

**resample fonksiyonu** kullanılarak 5'er dakikalık olacak şekilde veri yeniden şekillendirildi. Her **5dk'lık** dilim içerisindeki **son data** alındı. Bu, ilk parametre **5Min** ayarlanarak ve **last()** fonksiyonu kullanılarak yapıldı. 
>resample fonksiyonunda kullanılabilecek diğer parametreler
<br>

```	
    **Alias** 	**Description**
	B 		      business day frequency
	C 		      custom business day frequency (experimental)
	D 		      calendar day frequency
	W 		      weekly frequency
	M 		      month end frequency
	BM 		      business month end frequency
	CBM 	      custom business month end frequency
	MS 		      month start frequency
	BMS 	      business month start frequency
	CBMS 	      custom business month start frequency
	Q 		      quarter end frequency
	BQ 		      business quarter endfrequency
	QS 		      quarter start frequency
	BQS 	      business quarter start frequency
	A 		      year end frequency
	BA 		      business year end frequency
	AS 		      year start frequency
	BAS 	      business year start frequency
	BH 		      business hour frequency
	H 		      hourly frequency
	T, min 	      minutely frequency
	S 		      secondly frequency
	L, ms 	      milliseonds
	U, us 	      microseconds
	N 		      nanoseconds
```
Son veriyi değil de ortalama almak istersek **.last()** yerine **.mean()** fonksiyonu kullanılacak.
<br>
Benzer şekilde kullanılabilecek fonksiyonlar **max,min,std,sum,median,...**

In [ ]:
# Her bir gün ayrıca 5'er dakikaya ayrılarak csv formatında yazdırılıyor
# Yukarıda comment ile kapatılmış kodlar bu döngü içerisinde kullanıldı
for group in list(rd.groups):
    current_data = daily_data.get_group(group)
    current_data = current_data.resample('5Min',label='right',closed='right').last()
    current_data.to_csv(str(group.date())+'.csv')    

In [ ]:
# start_time = data.date_time.min()
# end_time = data.date_time.max()
# time_series = pd.date_range(start=start_time, end=end_time,freq='S')
# pivot_data = data.reindex(time_series)